In [7]:
import random
import torch
import itertools
import numpy as np
import editdistance

from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
from transformers import logging
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

logging.set_verbosity_error()

model_name_encoder = "allegro/herbert-base-cased"
device = 'cuda'

### feature extractor

In [19]:
tokenizer_encoder = AutoTokenizer.from_pretrained(model_name_encoder)
model_encoder = AutoModel.from_pretrained(model_name_encoder).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer_encoder(text, return_tensors='pt')['input_ids'][0]

print ([tokenizer_encoder.decode(idx) for idx in token_ids])

['<s>', 'Bardzo', 'lubię', 'lody', 'mali', 'nowe', 'z', 'bi', 'tą', 'śmietan', 'ą', '.', '</s>']


In [31]:
def representation(L):
    txt = ' '.join(L)
    input_ids = tokenizer_encoder(txt, return_tensors='pt')['input_ids'].to(device)
    output = model_encoder(input_ids=input_ids)
    return output.last_hidden_state.detach().cpu().numpy()[0,0,:]

In [5]:
representation(['Ala', 'ma', 'kota'])

array([ 7.75123611e-02, -1.26742795e-01,  1.28273174e-01,  6.88974857e-02,
        1.63994581e-02,  5.44659421e-02, -3.27508241e-01, -6.34695232e-01,
        9.20319483e-02,  8.05968642e-01,  1.59045756e-01,  2.29028508e-01,
        2.93846512e+00,  3.55168164e-01,  1.10074371e-01, -1.78765461e-01,
        1.20000631e-01,  7.92294275e-03, -1.32740542e-01,  2.68556684e-01,
        2.42710307e-01,  4.63390350e-02,  4.08300338e-03, -2.29847357e-01,
        1.29359066e-01,  2.12778866e-01,  2.44441584e-01,  8.24062228e-02,
        3.90837461e-01,  3.85150939e-01, -3.88434716e-02, -3.29763219e-02,
        6.87328950e-02, -4.60863337e-02,  1.64549291e-01, -5.30537553e-02,
       -2.92028993e-01, -7.47865140e-02, -3.22981179e-01,  2.10616335e-01,
       -4.49958332e-02, -2.33092010e-01, -1.98450804e-01,  4.04942811e-01,
        2.84321094e-03,  1.94603622e-01, -1.16448723e-01,  9.03226808e-02,
       -9.03956145e-02, -6.43575750e-03,  5.03770299e-02, -2.66933143e-01,
        1.24548480e-01, -

### manaul spoil func

In [ ]:
def spoil_maualy(L):
    replacements = {'ł': 'l', 'ą': 'a', 'ć': 'c', 'ę': 'e', 'ń': 'n', 'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z'}
    res = []
    for w in L:
        temp = random.random()
        if temp < 0.25:
            res.append(w.upper())
        elif temp < 0.50:
            letter_iondex = random.randrange(0, len(w))
            new_letter = random.choice('abcdefghijklmnopqrstuvwxyz')
            w = w[:letter_iondex] + new_letter + w[letter_iondex+1:]
            res.append(w)
        elif temp < 0.75:
            w = ''.join(replacements.get(c, c) for c in w)
            res.append(w)
        else:
            res.append(w)
    return res

In [43]:
spoil_maualy(['Ala', 'ma', 'kota'])

['ALA', 'oa', 'kota']

In [23]:
model_decoder_name = 'eryk-mazus/polka-1.1b'
tokenizer_decoder = AutoTokenizer.from_pretrained(model_decoder_name)
model_decoder = AutoModelForCausalLM.from_pretrained(model_decoder_name).to(device)

### decoder spoil func

In [ ]:
def spoil_with_decoder(L):
    prompt = f"""Zamień podaną opinię na nową, używając innych słów niż w oryginalnej opinii oraz zachowując przy tym wydźwięk i sens. Podkreśl wydźwięk oryginalnej opinii w zmienionej opinii. Niech opinia będzie krótkie.

Opinia: Bardzo lubię lody malinowe z bitą śmietaną.
Zamieniona opinia: Uwielbiam jeść lody malinowe z pyszną bitą śmietaną.

Opinia: {' '.join(L)}
Zamieniona opinia: """
    input_ids = tokenizer_decoder(prompt, return_tensors='pt')['input_ids'].to(device)
    output = model_decoder.generate(input_ids, max_length=300, num_return_sequences=1, do_sample=True)
    output_text = tokenizer_decoder.decode(output[0], skip_special_tokens=True)
    return output_text.replace(prompt, '')

In [ ]:
spoil_with_decoder('Te ciasteczka z posypką są naprawdę dobre.'.split())

'ładne desery są smaczne.'

### word2vec spoil func

https://dsmodels.nlp.ipipan.waw.pl/

5,7G


In [9]:
word2vec_path = '/home/patryk/Downloads/nkjp+wiki-forms-all-300-cbow-hs.txt'
# model = Word2Vec.load(word2vec_path)
model = KeyedVectors.load_word2vec_format(word2vec_path, binary=False, no_header=False)


In [26]:
def spoil_with_word2vec(L):
    res = []
    for w in L:
        w = "".join(filter(lambda x: "" if x in {'-', '_', '!', '?', '.', ','} else x, w.lower()))
        temp = []
        try:
            for w0, v in model.most_similar(w, topn=3):
                temp.append((w0, round(v, 2), editdistance.eval(w[-4::], w0[-4::]))) # (wrod, similarity, edit distance)
            temp.sort(key=lambda x: (x[2], -x[1]))
            res.append(temp[0][0])
        except KeyError:
            res.append(w)
        
    return res

In [27]:
spoil_with_word2vec('Polecam wszystkim tego fizjoterapeutę!'.lower().split())

['zalecam', 'wszytkim', 'tamtego', 'farmaceutę']

### create sets

In [28]:
def create_train_test(spoil_function, K=3):
    lines = open('../p1/reviews_for_task3.txt').readlines()
    random.shuffle(lines)

    N = len(lines)
    test_size = N // 4
    train_size = N - test_size

    train_lines = lines[:train_size]
    test_lines = lines[train_size:]

    X_train = []
    y_train = []
    X_test = []
    y_test = []

    for line in tqdm(train_lines):
        L = line.split()
        y = 0 if L[0] == 'BAD' else 1
        x = representation(L[1:])

        y_train.append(y)
        X_train.append(x)
        
        for i in range(K):
            x = representation(spoil_function(L[1:]))
            y_train.append(y)
            X_train.append(x)
        
        # if len(X_train) % 100 == 0:
        #     print (len(X_train))

    for line in tqdm(test_lines):
        L = line.split()
        y = 0 if L[0] == 'BAD' else 1
        
        x = representation(L[1:])
        y_test.append(y)
        X_test.append(x)
            
        # if len(X_test) % 100 == 0:
        #     print (len(X_test))
    return X_train, y_train, X_test, y_test

In [29]:
N = len(open('../p1/reviews_for_task3.txt').readlines())
test_size = N // 4
train_size = N - test_size

### testing

without spoiling

In [8]:
X_train, y_train, X_test, y_test = create_train_test(None, K=0)

100%|██████████| 100/100 [00:01<00:00, 74.80it/s]


In [9]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

print ('Train accuracy:', clf.score(X_train, y_train))
print ('Test accuracy:', clf.score(X_test, y_test))

Train accuracy: 1.0
Test accuracy: 0.75


manual

In [ ]:
X_train, y_train, X_test, y_test = create_train_test(spoil_maualy, K=3)

100%|██████████| 100/100 [00:01<00:00, 76.43it/s]


In [ ]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

print ('Train accuracy:', clf.score(X_train, y_train))
print ('Test accuracy:', clf.score(X_test, y_test))

Train accuracy: 0.9725
Test accuracy: 0.83


decoder

In [ ]:

X_train, y_train, X_test, y_test = create_train_test(spoil_with_decoder, K=3)

100%|██████████| 100/100 [00:01<00:00, 71.49it/s]


In [127]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

print ('Train accuracy:', clf.score(X_train, y_train))
print ('Test accuracy:', clf.score(X_test, y_test))

Train accuracy: 0.85
Test accuracy: 0.76


word2vec

In [32]:
X_train, y_train, X_test, y_test = create_train_test(spoil_with_word2vec, K=3)

100%|██████████| 100/100 [00:01<00:00, 66.63it/s]


In [33]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

print ('Train accuracy:', clf.score(X_train, y_train))
print ('Test accuracy:', clf.score(X_test, y_test))

Train accuracy: 0.9833333333333333
Test accuracy: 0.77
